In [2]:
import pandas as pd

import matplotlib.pyplot as plt

import plotly
import plotly.express as px
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

In [3]:
df= pd.read_csv("C:/Users/manis/Downloads/Aus_bushfire.csv")

df = df[df["Country"] == "AU"]
df["date"] = pd.to_datetime(df["acq_date"], dayfirst=True)
df["month"] = df["date"].dt.month_name(locale="English")
df['day'] = pd.DatetimeIndex(df['date']).day
df.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,confidence,version,...,daynight,new lat,new long,Sub Urban,State,Region,Country,date,month,day
0,-42.12143,147.96153,304.2,0.33,0.56,4/12/2019,14:12,1,nominal,1.0NRT,...,N,-42.78161,147.56267,Sorell,Tasmania,Sorell,AU,2019-12-04,December,4
1,-42.12246,147.96536,300.6,0.33,0.56,4/12/2019,14:12,1,nominal,1.0NRT,...,N,-42.78161,147.56267,Sorell,Tasmania,Sorell,AU,2019-12-04,December,4
2,-42.12349,147.96916,307.2,0.33,0.56,4/12/2019,14:12,1,nominal,1.0NRT,...,N,-42.78161,147.56267,Sorell,Tasmania,Sorell,AU,2019-12-04,December,4
3,-42.12545,147.97643,297.8,0.33,0.56,4/12/2019,14:12,1,nominal,1.0NRT,...,N,-42.78161,147.56267,Sorell,Tasmania,Sorell,AU,2019-12-04,December,4
4,-42.12209,147.94337,296.0,0.34,0.56,4/12/2019,14:12,1,nominal,1.0NRT,...,N,-42.78161,147.56267,Sorell,Tasmania,Sorell,AU,2019-12-04,December,4


In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

month_filter = dbc.Card(html.Div([
                            dbc.CardHeader("Choose Month", style={'text-align': 'center', 'color': 'white', 'backgroundColor':'#313333'}),
                            dbc.CardBody(dcc.Dropdown(
                                id='month-dropdown',
                                options=[
                                    {'label': 'December', 'value': 'December'},
                                    {'label': 'January', 'value': 'January'},
                                    {'label': 'February', 'value': 'February'},
                                    {'label': 'March', 'value': 'March'},
                                    {'label': 'April', 'value': 'April'}
                                        ],
                                value='December')),
                                ])
                        )

day_filter = dbc.Card(html.Div([
                        dbc.CardHeader("Choose Day", style={'text-align': 'center', 'color': 'white', 'backgroundColor':'#313333'}),
                        dbc.CardBody(dcc.Slider(
                            id='day-slider',
                            min=1,
                            max=31,
                            value=4,
                            marks={str(day): str(day) for day in df['day'].unique()},
                            step=1,
                            updatemode="drag")), 
                                ])
                     )

territory_filter = dbc.Card(html.Div([
                                dbc.CardHeader("Choose Territory", style={'text-align': 'center', 'color': 'white', 'backgroundColor':'#313333'}),
                                dbc.CardBody(dcc.Dropdown(
                                    id='territory-dropdown',
                                    options=[{'label': i, 'value': i} for i in df["State"].unique()],
                                    value='Tasmania')),

                                    ])
                           )

frp_density = dbc.Card([
                dbc.CardHeader("Fire Power Radiation", style={'text-align': 'center', 'color': 'white', 'backgroundColor':'#313333'}),
                dbc.CardBody(
                    html.Div([
                        dcc.Graph(id='frp-heat-map', config = {'displayModeBar': False})
                            ])
                            )
                      ])

bright_density = dbc.Card([
                dbc.CardHeader("Bightness Index", style={'text-align': 'center', 'color': 'white', 'backgroundColor':'#313333'}),
                dbc.CardBody(
                    html.Div([
                        dcc.Graph(id='bright-heat-map', config = {'displayModeBar': False})
                            ])
                            )
                         ])

bar_graph = dbc.Card([
                dbc.CardHeader("Region-wise Max Fire Power Radiation", style={'text-align': 'center', 'color': 'white', 'backgroundColor':'#313333'}),
                dbc.CardBody(html.Div([
                                        dcc.Graph(id='bar-graph', config = {'displayModeBar': False})
                                      ])
                             )
                    ])

trend_graph = dbc.Card([
                dbc.CardHeader("Fire Power Radiation Trend", style={'text-align': 'center', 'color': 'white', 'backgroundColor':'#313333'}),
                dbc.CardBody(html.Div([
                                        dcc.Graph(id='trend-graph', config = {'displayModeBar': False})
                                      ])
                             )
                    ])

app.layout = html.Div([
    dbc.Row(dbc.Col(html.H4("..............Australian Bushfire............."),
                width= {"size": 6, "offset": 3}
                   ), justify="center", align="center"),
    dbc.Row([dbc.Col(month_filter, width= 4), 
             dbc.Col(day_filter, width= 4),
             dbc.Col(territory_filter, width= 4)], style = {"padding": "0rem 1rem 0.5rem 1rem"}),
    dbc.Row([dbc.Col(frp_density, width = 6),
             dbc.Col(bright_density, width = 6)], style = {"padding": "0.1rem 1rem 0.5rem 1rem"}),
    dbc.Row([dbc.Col(bar_graph, width = 6),
             dbc.Col(trend_graph, width = 6)], style = {"padding": "0.1rem 1rem 0.5rem 1rem"})
])

@app.callback(
    Output('frp-heat-map', 'figure'),
    [Input('month-dropdown', 'value'),
     Input('day-slider', 'value')])
def frp_density_graph(selected_month, selected_day):
    filtered_df = df[(df["month"] == str(selected_month)) & (df["day"] == int(selected_day))]
    fig = px.density_mapbox(filtered_df, lat='latitude', lon='longitude', z="frp", radius=10,
                            center=dict(lat=-24.509671, lon=133.619221), zoom=2.5,
                            mapbox_style="stamen-terrain")
    fig.update_layout(autosize=False, width=700, height=200, margin=dict(l=0, r=0, b=0, t=0, pad=0))

    return fig

@app.callback(
    Output('bright-heat-map', 'figure'),
    [Input('month-dropdown', 'value'),
     Input('day-slider', 'value')])
def bright_density_graph(selected_month, selected_day):
    filtered_df = df[(df["month"] == str(selected_month)) & (df["day"] == int(selected_day))]
    fig = px.density_mapbox(filtered_df, lat='latitude', lon='longitude', z="bright_ti4", radius=10,
                            center=dict(lat=-24.509671, lon=133.619221), zoom=2.5,
                            mapbox_style="stamen-terrain")
    fig.update_layout(autosize=False, width=700, height=200, margin=dict(l=0, r=0, b=0, t=0, pad=0))
    
    return fig

@app.callback(
    Output('bar-graph', 'figure'),
    [Input('month-dropdown', 'value'),
     Input('territory-dropdown', 'value')])
def bar_graph(selected_month, selected_territory):
    filtered_df = df[(df["month"] == str(selected_month)) & (df["State"] == selected_territory)]
    grouped = filtered_df.groupby("Region").agg(max_frp = pd.NamedAgg(column = "frp", aggfunc = "max")).reset_index()
    grouped.sort_values(by = "max_frp", inplace = True, ascending = False)
    grouped = grouped.iloc[0:5, :]
    fig = px.bar(grouped, x="Region", y="max_frp", color='Region')
    fig.update_layout(autosize=False, width=700, height=160, margin=dict(l=0, r=0, b=0, t=0, pad=0), 
                      xaxis = dict(showgrid=False, showticklabels=False), 
                      paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    fig.update_yaxes(title_text='Max FRP')
    
    return fig

@app.callback(
    Output('trend-graph', 'figure'),
    [Input('month-dropdown', 'value'),
     Input('territory-dropdown', 'value')])
def trend_graph(selected_month, selected_territory):
    filtered_df = df[(df["month"] == str(selected_month)) & (df["State"] == str(selected_territory))]
    grouped = filtered_df.groupby(["Region", "day"]).agg(max_frp = pd.NamedAgg(column = "frp", aggfunc = "mean")).reset_index()
    fig = px.line(grouped, x="day", y="max_frp", color='Region')
    fig.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 1, dtick = 1, showgrid=True),
                      autosize=False, width=700, height=160, margin=dict(l=0, r=0, b=0, t=0, pad=0),
                      paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    fig.update_traces(textfont_color='white')
    fig.update_yaxes(title_text='Average FRP')
    fig.update_xaxes(title_text='Day')
    
    return fig

if __name__ == '__main__':
     app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jun/2020 00:18:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:18:21] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:18:21] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:19:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:19:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:19:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:19:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:20:20] "POST /_dash-update-component

127.0.0.1 - - [26/Jun/2020 00:23:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Jun/2020 00:23:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.